# BAT.jl Tutorial - Poisson Counting Experiment

Note: This tutorial is written in the style of a cloze text. Parts that have to be filled out by you are marked `#=FILLME=#`.

The [the tutorial section](https://bat.github.io/BAT.jl/stable/tutorial/) of the BAT.jl documentation should prove helpful.

In [ ]:
using BAT
using Distributions 
using IntervalSets
using ValueShapes
using Plots
using ArraysOfArrays
using StatsBase 
using LinearAlgebra

## Situation
We want to determine the properties of a radioactive singal source in the presence of background from natural sources of radioactivity.
We assume to have one signal source $S$ and only one source of background $B$.

## 1. Background only measurement
We start by using our detector without the signal source installed.
This measurement yields a number of $k_B=10$ counts.  
Using this measurement we want to gain information about the event rate of the natural radioactive background.

### Task: 
Perform a Bayesian analysis to estimate the event rate of the natural background $\lambda_b$ using a Poisson model. 

Start by defining the log-likelihood function, using the function `logpdf()` and type `Poisson` provided by the package [Distributions.jl](https://juliastats.github.io/Distributions.jl/latest/univariate/):

In [ ]:
# Number of observed background events
kb = 10

likelihood_B = let k = kb
    params -> begin
        return logpdf(Poisson(params.λb), k) # poisson log-likelihood
    end
end;

Define the Prior with help of the `NamedTupleDist()` function. Use a flat prior between 0 and 30:

In [ ]:
prior_B = NamedTupleDist(
    λb = #=FILLME=#
)

Then use the likelihood and the prior to define the `PosteriorDensity()`:

In [ ]:
posterior_B = PosteriorDensity(#=FILLME=#); 

Define the settings for the sampling. Choose `MetropolisHastings()` as the algorithm and set the number of chains and samples (4 chains with 10^5 samples each should be fine):

In [ ]:
algorithm = #=FILLME=#)
nchains = #=FILLME=#)
nsamples = =#=FILLME=#)

Use the function `bat_sample` to sample the posterior using `nchains` MCMC chains, generating `nsamples` per chain:

In [ ]:
samples_B, stats_B = bat_sample(posterior_B, (nsamples, nchains), algorithm)
stats_B

Take a look at the resulting disribution for the background rate using `plot()`:

In [ ]:
plot(posterior_B, samples_B, :λb)
plot!(prior_B, :λb, xlabel = "\$\\lambda_b\$", ylabel = "\$P(\\lambda_b)\$")

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_B.mode)")
println("Mean: $(stats_B.mean)")
println("Covariance: $(stats_B.cov)")
println("Standard Deviation: $(diag(sqrt(stats_B.cov[:, :])))")

## 2. Second background only measurement
A second measurement of the natural background yields a number of $k_B=8$ counts.  
Therefore, we want to update our estimation of the background rate using this new knowledge together with the pervious results.
### Task:
Perform an analysis of the new measurement similar to the first one.   
Use the posterior distribution of the previous background measurement as a prior for this analysis.  
This can be done by using a [StatsBase histogram](http://juliastats.github.io/StatsBase.jl/latest/empirical/#Histograms-1) of the samples: `fit(Histogram, flatview(samples)[i,:], weights, nbins)`.  
The weights of the samples need to be used with the `FrequencyWeights()` function.

Define a `StatsBase` histogram containing the previous posterior distribution:

In [ ]:
posterior_hist_B1 = fit(Histogram, flatview(samples_B.params)[1, :], FrequencyWeights(samples_B.weight), nbins = 400);

The histogram can be used as a prior by converting it into a univariate distribution using `BAT.HistogramAsUvDistribution(histogram)`.  
Apart from that, you can proceed similarly to the first task.

Define the log-likelihood function:

In [ ]:
kb2 = 8

likelihood_B2 = let k = kb2
    params -> #=FILLME=#
end

Define the prior (use the posterior of the previous task) and the posterior:

In [ ]:
prior_B2 = NamedTupleDist( #=FILLME=# )

posterior_B2 = #=FILLME=#

Generate samples:

In [ ]:
samples_B2, stats_B2 = #=FILLME=#
stats_B2

Use the  `plot(samples)` and `plot!(prior)` functions to visualize the posterior of the first analysis and the updated posterior together:

In [ ]:
plot(posterior_B2, #=FILLME=#)
plot!(prior_B2, #=FILLME=#)

Print some statistics of the samples:

In [ ]:
println("Mode: #=FILLME=#")
println("Mean: #=FILLME=#")
println("Covariance: #=FILLME=#")
println("Standard Deviation: #=FILLME=#")

## 3. Signal + Background
Including the radioactive source to the experimental setup, we repeat the measurement and obtain $k_{S+B}=12$ counts.
With this measurement and our prior knowledge about the background we are able to estimate the rate of the signal $\lambda_s$.
### Task
Perform a third analysis using a poisson model with the combined singal + background rate.
Use the known information about the background from the previous tasks as a prior and choose a suitable prior for the signal.

Define the likelihood for the signal + background model:

In [ ]:
# Number of observed events
kSB = 12

likelihood_SB = #=FILLME=#

Define the prior for both the signal and backgound parameters.  
Remember to use the knowledge from the previous tasks for the background. (hint: `BAT.HistogramAsUvDistribution()`)  


In [ ]:
hist_B2 = #=FILLME=#
B2 = #=FILLME=#

prior_SB = #=FILLME=#

Also define the posterior:

In [ ]:
posterior_SB = #=FILLME=#

Generate samples for the signal + background model:

In [ ]:
samples_SB = #=FILLME=#
stats_SB

To visualize an overview of the results for both prameters use `plot(samples)`.   
(Hint: use the keyword `param_labels=["\\lambda_b","\\lambda_s"]` for correct axis labels)

In [ ]:
plot(samples_SB, #=FILLME=#)

Print some statistics of the samples:

In [ ]:
println("Mode: #=FILLME=#")
println("Mean: #=FILLME=#")
println("Covariance: #=FILLME=#")
println("Standard Deviation: #=FILLME=#"")

## 4. Error propagation

Finally, we want to caluclate the cross section of the signal process.
The rate of measured events in the detector of a couting experiment can be written as 

$\frac{\mathrm d N}{\mathrm d t} = \epsilon \cdot σ \cdot L$ ,

with the Luminosity $L$ and the efficiency of the detector $\epsilon$.   
The signal cross section is therefore given as:
### $σ_S = \frac{λ_s}{\epsilon \cdot L}$  .
For this experiment we assume a luminosity $L = 1.1$  (neglecting units).

As a final result we want to obtain either a measurement or an upper limit on the signal cross section.

### Task 4 a) Known efficiency with gaussian uncertainty
The detector efficiency has been measured to be $\epsilon = 0.1 \pm 0.02$, assuming the uncertainties to follow a normal distribution.

Calculate the signal cross section $σ_S$ using the equation above.

Use the [Distrtibutions.jl](https://juliastats.github.io/Distributions.jl/latest/univariate/) package and `rand()` to obain a sample for $\epsilon$.  
In order to obtain unweigthed samples of $\lambda_S$ by resampling the posterior samples, the function `bat_sample(samples, nsamples).samples` can be used.

The function `broadcast()` or the `.`operator (e.g. `a .+ b`) might be useful for element-wise operation when handeling the samples.  

Define the luminosity and the efficiency:

In [ ]:
L = #=FILLME=#
ϵ = #=FILLME=#

Plot the efficiency.  
Use a [StatsBase histogram](http://juliastats.github.io/StatsBase.jl/latest/empirical/#Histograms-1) to visualize the distribution.  
(Hint: The plot recipes can also be used for the `StatsBase` histograms) 

In [ ]:
hist_ϵ = fit(Histogram, ϵ, nbins=200)
#!EX hist_ϵ = fit(#=FILLME=#)

plot(hist_ϵ, 1, seriestype=:smallest_intervals, xlabel="\$\\epsilon\$", ylabel="\$p(\\epsilon)\$")

Get unweighted samples for the signal rate and calculate the cross section distribution:

In [ ]:
resampled_SB = bat_sample(samples_SB, nsamples).samples;
λ_SB = valshape(prior_SB).(resampled_SB.params).λs
σS = (λ_SB)./(ϵ*L);

Plot the distribution of the signal cross section. (Hint: use again a histogram)  
From the plot, determine the 95% upper limit on the cross section.

In [ ]:
hist_σ = #=FILLME=#
plot(hist_σ, #=FILLME=#)

### Task 4 b) Binomial analysis of calibration measurement to determine efficiency 
We now want to perform the same analysis as in task 4 a) but for the case that the detector efficiency $\epsilon$ is not yet known.   

 Instead, $\epsilon$ is to be determined using a calibration measurement with a source for which the signal rate is known. Then it is possible to calculate the efficiency from the expected number of counts and the number of counts actually measured with the detector.   


For our example, the number of expected events is assumed to be $N_\text{expected} = 200$.  
The detector measures only $N_\text{measured} = 21$ events.  

Task: Implement a binomial model using the `Binomial(n,p)` function of the [Distrtibutions.jl](https://juliastats.github.io/Distributions.jl/latest/univariate/) package and determine the distribution of the detector efficiency.  
Afterwards, repeat the steps from 4 a) using the obtained distribution of the efficiency to calculate the signal cross section .

Define the binomial likelihood:

In [ ]:
n_expected = 200
n_measured = 21

likelihood_binomial = let #=FILLME=#
    params -> #=FILLME=#
end

Define the prior (flat) for the efficiency and define the posterior:

In [ ]:
prior_binomial = #=FILLME=#

posterior_binomial = #=FILLME=#

Generate the samples:

In [ ]:
samples_binomial, stats_binomial = #=FILLME=#
stats_binomial

Plot the distribution of the efficiency:  
(What do you observe when comparing to the efficiency used in 4 a) ?)

In [ ]:
plot(samples_binomial, #=FILLME=#)

Print some statistics of the samples:

In [ ]:
println("Mode: $(stats_binomial.mode)")
println("Mean: $(stats_binomial.mean)")
println("Covariance: $(stats_binomial.cov)")
println("Standard Deviation: $(diag(sqrt(stats_binomial.cov[:, :])))")

In [ ]:
resampled_SB = #=FILLME=#
resampled_binomial = #=FILLME=#
length(resampled_SB), length(resampled_binomial)

In [ ]:
λ_SB = valshape(prior_SB).(resampled_SB).params.λs
ϵ   = #=FILLME=#
σS  = #=FILLME=#

Use a `StatsBase`histogram to visualize the cross section distribution.  
From the plot, determine the 95% upper limit on the cross section.

In [ ]:
hist_σ = fit(Histogram, σS, nbins=200)
plot(hist_σ, 1, seriestype=:smallest_intervals, xlabel="\$\\sigma_s\$", ylabel="\$P(\\sigma_s)\$")

Compare the distribution of the signal cross section to the distribution from 4 a).